In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-kucamyss/unsloth_3e35c043f44c48e8a2d88712069840be
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-kucamyss/unsloth_3e35c043f44c48e8a2d88712069840be
  Resolved https://github.com/unslothai/unsloth.git to commit 8b80820b8b9f13ab4ecca089ec6ff92c58530bea
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 10.1 MB/s eta 0:00:00
   ━━

In [ ]:
from unsloth import FastLanguageModel

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
#     "unsloth/gemma-7b-it-bnb-4bit",
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

In [ ]:
model.generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "max_length": 131072,
  "pad_token_id": 128004,
  "temperature": 0.6,
  "top_p": 0.9
}

In [ ]:
import re
def predict (inp_text):
      messages = [
          {"from": "human", "value": f"{inp_text}"},
      ]
      inputs = tokenizer.apply_chat_template(
          messages,
          tokenize = True,
          add_generation_prompt = True, # Must add for generation
          return_tensors = "pt",
      ).to("cuda")
      model.generation_config.pad_token_id = tokenizer.pad_token_id
      outputs = model.generate(input_ids = inputs, use_cache = True ,temperature = 0.1)
      result = tokenizer.batch_decode(outputs)
      # print(result)
      return filter_user_assistant_msgs(result[0])

def filter_user_assistant_msgs(text):
  # Match the first user message
  # print(text)
  msg_pattern = r".*>assistant\n(.*?)<\|im_end\|>"
  match = re.match(msg_pattern, text, re.DOTALL)
  if match:
    message = match.group(1).strip()
  else:
    message = text
  return message

In [ ]:
 predict("आधार कैसे अपडेट करें")

'आपका आधार नंबर अपडेट करने के लिए, निम्नलिखित चरणों का पालन करें:\n\n1. **आधार कार्ड की प्रति का प्रिंट आउट लें**: अपने आधार कार्ड की प्रति का प्रिंट आउट लें और सुनिश्चित करें कि यह साफ और स्पष्ट है।\n\n2. **आधार कार्ड की प्रति को संबंधित अधिकारी के पास ले जाएं**: अपने आधार कार्ड की प्रति को संबंधित अधिकारी के पास ले जाएं, जैसे कि बैंक, पोस्ट ऑफिस, या किसी अन्य संस्थान के पास जो आपके आधार कार्ड को अपडेट करने की सुविधा प्रदान करते हैं।\n\n3. **आधार कार्ड की प्रति को संबंधित अधिकारी के पास जमा करें**: संबंधित अधिकारी के पास जाने के बाद, अपने आधार कार्ड की प्रति को जमा करें और अपडेट के लिए अनुरोध करें।\n\n4. **आधार कार्ड की प्रति को अपडेट करने के लिए आवश्यक दस्तावेज प्रदान करें**: यदि आपके आधार कार्ड में कोई परिवर्तन है, तो आवश्यक दस्तावेज प्रदान करें, जैसे कि पहचान पत्र, पते का प्रमाण, या किसी अन्य आवश्यक दस्तावेज की प्रति।\n\n5. **आधार कार्ड की प्रति को अपडेट करने के लिए शुल्क का भुगतान करें**: यदि आवश्यक हो, तो आधार कार्ड की प्रति को अपडेट करने के लिए शुल्क का भुगतान करें।\n\n6. **आधार